In [1]:
import codecs
import re
import pandas as pd
import requests
import json
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
URL = "https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:1/Y:1/station:0"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'Accept': 'application/json',
    'Referer': 'https://www.marinetraffic.com/',
}
try:
    response = requests.get(URL,headers=headers)
    response.raise_for_status
    print("Status Code:", response.status_code)
    print("Response Text:", response.text[:500])

    data = response.json()

except requests.exceptions.RequestException as e:
    print("Abort THE MISSION CHIGGA",e)

Status Code: 403
Response Text: <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" 
Abort THE MISSION CHIGGA Expecting value: line 1 column 1 (char 0)


In [ ]:

# Set up Selenium with Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless to stay stealth
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36')

# Auto-install ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    # Step 1: Load main page to grab cookies and session
    print("Hitting main page to snag cookies...")
    driver.get("https://www.ambitionbox.com/list-of-companies?page=1")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located(("tag name", "body")))  # Wait for page load
    time.sleep(3)  # Extra buffer for JS/cookies

    # Grab cookies
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}
    print("Cookies grabbed:", cookies)

    # Step 2: Try to find Vessel-Token or other headers (if needed)
    # Check DevTools > Application > Local Storage or run JS to find tokens
    try:
        vessel_token = driver.execute_script("return localStorage.getItem('Vessel-Token') || ''")
        print("Vessel-Token:", vessel_token)
    except:
        vessel_token = None
        print("No Vessel-Token found in localStorage")

    # Headers for XHR request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
        'Accept': 'application/json',
        'X-Requested-With': 'XMLHttpRequest',
    }
    if vessel_token:
        headers['Vessel-Token'] = vessel_token  # Add if found

    # Step 3: Hit the XHR endpoint for a single tile
    base_url = "https://www.ambitionbox.com/list-of-companies?page=1"
    x, y = 1, 1  # Start with one tile
    url = base_url.format(x, y)

    print(f"Trying XHR URL: {url}")
    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        print("Response Text (first 500 chars):", response.text[:500])

        # Parse JSON
        data = response.json()
        ships = data.get('data', {}).get('rows', [])
        print(f"Got {len(ships)} ships from X:{x}, Y:{y}")

        # Convert to DataFrame
        df = pd.DataFrame(ships)
        print("DataFrame head:", df.head().to_string())

        # Save to CSV
        df.to_csv('ships_X1_Y1.csv', index=False, encoding='utf-8')
        print("Saved to ships_X1_Y1.csv")

    except requests.exceptions.RequestException as e:
        print("XHR failed, damn it:", e)
        print("Response Text (if any):", response.text[:500] if 'response' in locals() else "No response")
        raise  # Let's debug this first

    # Step 4: Loop through multiple tiles (uncomment once single tile works)
    """
    all_ships = []
    x_range = range(0, 5)  # Adjust based on zoom/area
    y_range = range(0, 5)
    for x in x_range:
        for y in y_range:
            url = base_url.format(x, y)
            try:
                response = requests.get(url, headers=headers, cookies=cookies)
                response.raise_for_status()
                data = response.json()
                ships = data.get('data', {}).get('rows', [])
                all_ships.extend(ships)
                print(f"Got {len(ships)} ships from X:{x}, Y:{y}")
                time.sleep(2)  # Avoid banhammer
            except requests.exceptions.RequestException as e:
                print(f"Failed for X:{x}, Y:{y}: {e}")
                continue

    # Save all ships
    df_all = pd.DataFrame(all_ships)
    df_all.to_csv('all_ships.csv', index=False, encoding='utf-8')
    print(f"Total ships collected: {len(all_ships)}, saved to all_ships.csv")
    """

except Exception as e:
    print("Mission blown to shit:", e)

finally:
    driver.quit()  # Clean up


Hitting main page to snag cookies...
Cookies grabbed: {'_ga': 'GA1.1.811175824.1758298377', '_ga_HV7DJVVBCW': 'GS2.1.s1758298377$o1$g0$t1758298377$j60$l0$h0', '_gcl_au': '1.1.2130086513.1758298377', '_t_ds': '181e55031758298372-40181e5503-0181e5503', 'bm_sv': '967CEA1CC60A6A883ADB41A2722CED83~YAAQpwzVF/Zm00yZAQAAtQbAYh2EoSA1vDcxpH3j8Ug7Bb2VjCKjxAyd3ShYnSjq/HvThOZUGzuDbst1Trzc+DoPEdpyMVFl1it6eikJpioESlHxjI3s4nWXHfAop78zU/z+4oYKaAryOCOeHs293YhFeVMDZjA5kp1+ZbkjaVVQgisqkUVzhmncvi61tO7kymnZUIQS3mebZhH+jj58ROdMl3xvtQHEtCB2RGFk3TkJ7S2VsKO97y127iFTNDfDNNXZ0VA=~1', '_fbp': 'fb.1.1758298377897.544336355560455878', 'ak_bmsc': '5558A6185220D0B5AB2C69B7360A57DE~000000000000000000000000000000~YAAQpwzVF8Bm00yZAQAArQLAYh0HQNBzyRIuS6PVtogqQXFDken0iKfJMBy6ZHS9+Tulenb6YpYJaRkFBBGfqKtZT4xOp57NEjyTDZtiL04r7kq4zJEleJWvF/+t5enpdg/Y2L3Zfntggqd6Pw0yMaZzclzMVu5Nb7mWx1zFJscfQldtStr5fyDrqNA+/bObp7fBFkrdoO07CoVNbGGxLCa8w+pCHejOcm/O9XnWqQUpvdNo66qoTU4w6hQG6Opr4us/aD/PN5x20vwLwCsq4iYKXdQyYbFIhT7/flUJoXD2K56HUgsSJpmb

In [2]:


# Set up Selenium to dodge Cloudflare
options = webdriver.ChromeOptions()
 # Headless for stealth
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-blink-features=AutomationControlled')  # Hide Selenium
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36')
# Mimic real browser
options.add_argument('--window-size=1920,1080')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

# Auto-install ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    # Step 1: Load main page to pass Cloudflare and grab cookies
    print("Hitting main page to bypass Cloudflare...")
    driver.get("https://www.marinetraffic.com/en/ais/home")
    WebDriverWait(driver, 15).until(EC.presence_of_element_located(("tag name", "body")))
    time.sleep(5)  # Wait for JS, cookies, and Cloudflare checks

    # Check if Cloudflare blocked us
    page_source = driver.page_source
    if "Cloudflare" in page_source or "Access denied" in page_source:
        print("Cloudflare’s being a bitch. Page source:", page_source[:500])
        raise Exception("Cloudflare block detected")

    # Grab cookies
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}
    print("Cookies grabbed:", cookies)

    # Step 2: Hunt for Vessel-Token or other auth tokens
    try:
        vessel_token = driver.execute_script("return localStorage.getItem('Vessel-Token') || ''") or \
                       driver.execute_script("return sessionStorage.getItem('Vessel-Token') || ''")
        print("Vessel-Token:", vessel_token)
    except:
        vessel_token = None
        print("No Vessel-Token found in localStorage/sessionStorage")

    # Step 3: Try XHR with requests
    base_url = "https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:{}/Y:{}/station:0"
    x, y = 1, 1
    url = base_url.format(x, y)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
        'Accept': 'application/json',
        'Referer': 'https://www.marinetraffic.com/',
        'X-Requested-With': 'XMLHttpRequest',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    if vessel_token:
        headers['Vessel-Token'] = vessel_token

    print(f"Trying XHR URL with requests: {url}")
    try:
        response = requests.get(url, headers=headers, cookies=cookies)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        print("Response Text (first 500 chars):", response.text[:500])

        # Parse JSON
        data = response.json()
        ships = data.get('data', {}).get('rows', [])
        print(f"Got {len(ships)} ships from X:{x}, Y:{y}")

        # Save to DataFrame and CSV
        df = pd.DataFrame(ships)
        print("DataFrame head:", df.head().to_string())
        df.to_csv('ships_X1_Y1.csv', index=False, encoding='utf-8')
        print("Saved to ships_X1_Y1.csv")

    except requests.exceptions.RequestException as e:
        print("Requests XHR failed:", e)
        print("Response Text (if any):", response.text[:500] if 'response' in locals() else "No response")

        # Step 4: Fallback to Selenium for XHR
        print("Falling back to Selenium for direct XHR...")
        driver.get(url)  # Try loading the XHR URL directly
        time.sleep(3)
        page_source = driver.page_source
        if "Cloudflare" in page_source:
            print("Cloudflare blocked Selenium too:", page_source[:500])
            raise Exception("Cloudflare block on Selenium XHR")

        # Try parsing JSON from page source
        try:
            # Extract JSON from <pre> tag or raw response
            soup = BeautifulSoup(page_source, 'html.parser')
            pre_tag = soup.find('pre')
            if pre_tag:
                data = json.loads(pre_tag.text)
                ships = data.get('data', {}).get('rows', [])
                print(f"Selenium got {len(ships)} ships from X:{x}, Y:{y}")
                df = pd.DataFrame(ships)
                df.to_csv('ships_X1_Y1_selenium.csv', index=False, encoding='utf-8')
                print("Saved to ships_X1_Y1_selenium.csv")
            else:
                print("No JSON found in page source:", page_source[:500])
        except json.JSONDecodeError as je:
            print("JSON parse failed in Selenium:", je)
            print("Page source:", page_source[:500])

except Exception as e:
    print("Mission fucked:", e)

finally:
    driver.quit()


Hitting main page to bypass Cloudflare...
Cookies grabbed: {'cf_clearance': '_Nj5m_3AOr5u.3q4Qyd8lw7dCazyv..ktCHB72poD4g-1758298889-1.2.1.1-PcAi7_nui7k8439iNiqYNQq0r8K8yM8q4jSqyfawQRMKwF5al0qI0gtfveVqgovWnr_hYkgDKqBjAZYWDBXgV2UXeDbncq9D2lxaVxk09bU7Go1zLHYSPVA5O00XjkLy2IDUFmttcoxGrYuO3eJtaPxgrEvJ89zKj.SOMx7Bi1ndp2dBxvrg7DXxBHNCqKSRzf0umdTC8ywM01T2VofbNNHjFKXUmGKNOWaX6dghEFs', 'usprivacy': '1N--', '_cfuvid': 'WthmpDFR2HbTyIGpvTxXowkar7Bhf37BCZPdT_qbsyk-1758298887557-0.0.1.1-604800000', '__cf_bm': 'G5RwWMdJIgfR_UJjZmI7fXv9Mv6kqT.2Ba._tFfPLz4-1758298887-1.0.1.1-UA_rowpFbpZfIE_5m1HTQvv2c8.zqJDkNRbPu9p4tL6mCmAnCkI8atXHAIgg8KyuVws4MITCyp0GCHCCQcxgjbLVKML6wT3JH6ZQL5j9KTE'}
Vessel-Token: 
Trying XHR URL with requests: https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:1/Y:1/station:0
Requests XHR failed: 403 Client Error: Forbidden for url: https://www.marinetraffic.com/getData/get_data_json_4/z:3/X:1/Y:1/station:0
Response Text (if any): <!DOCTYPE html>
<!--[if lt IE 7]> <html class="